## CSCI 5654 - Linear Programming - Project

***
#### Team Members
#### 1. Ketan Ramesh
#### 2. Shreyas Gopalakrishna
***

## Vehicle Routing Problem

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import distance
import random
import pulp

In [376]:
numberOfCustomers = 5
capacityOfVehicle = 6
numberOfVehicles = 3
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {i: np.random.randint(1, 10) for i in C}
demandOfCustomers[0] = 0
demandOfCustomers[numberOfCustomers+1] = 0

In [377]:
#   [0 1 2 3 0]
# 0 [i 1 1 1 0]
# 1 [1 i 1 1 1]
# 2 [1 1 i 1 1]
# 3 [1 1 1 i 1]
# 0 [0 1 1 1 i]

In [378]:
# Creating random coordinates
xCoordinates = np.random.rand(len(V))*1000
yCoordinates = np.random.rand(len(V))*1000

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 999999
costMatrix

array([[9.99999e+05, 4.04000e+02, 3.07000e+02, 6.21000e+02, 1.18000e+02,
        3.38000e+02, 0.00000e+00],
       [4.04000e+02, 9.99999e+05, 3.30000e+02, 9.24000e+02, 5.23000e+02,
        7.33000e+02, 3.42000e+02],
       [3.07000e+02, 3.30000e+02, 9.99999e+05, 6.13000e+02, 3.96000e+02,
        5.24000e+02, 2.63000e+02],
       [6.21000e+02, 9.24000e+02, 6.13000e+02, 9.99999e+05, 5.66000e+02,
        3.91000e+02, 6.51000e+02],
       [1.18000e+02, 5.23000e+02, 3.96000e+02, 5.66000e+02, 9.99999e+05,
        2.34000e+02, 1.82000e+02],
       [3.38000e+02, 7.33000e+02, 5.24000e+02, 3.91000e+02, 2.34000e+02,
        9.99999e+05, 3.95000e+02],
       [0.00000e+00, 3.42000e+02, 2.63000e+02, 6.51000e+02, 1.82000e+02,
        3.95000e+02, 9.99999e+05]])

In [379]:
costMatrix2 = np.copy(costMatrix)

In [385]:
costMatrix2

array([[9.99999e+05, 4.04000e+02, 3.07000e+02, 6.21000e+02, 1.18000e+02,
        3.38000e+02, 0.00000e+00],
       [4.04000e+02, 9.99999e+05, 3.30000e+02, 9.24000e+02, 5.23000e+02,
        7.33000e+02, 3.42000e+02],
       [3.07000e+02, 3.30000e+02, 9.99999e+05, 6.13000e+02, 3.96000e+02,
        5.24000e+02, 2.63000e+02],
       [6.21000e+02, 9.24000e+02, 6.13000e+02, 9.99999e+05, 5.66000e+02,
        3.91000e+02, 6.51000e+02],
       [1.18000e+02, 5.23000e+02, 3.96000e+02, 5.66000e+02, 9.99999e+05,
        2.34000e+02, 1.82000e+02],
       [3.38000e+02, 7.33000e+02, 5.24000e+02, 3.91000e+02, 2.34000e+02,
        9.99999e+05, 3.95000e+02],
       [0.00000e+00, 3.42000e+02, 2.63000e+02, 6.51000e+02, 1.82000e+02,
        3.95000e+02, 9.99999e+05]])

In [381]:
len(costMatrix)
print(costMatrix[0][1]+costMatrix[1][3]+costMatrix[3][2]+costMatrix[2][4])

2337.0


In [9]:
# PuLP class for vehicle routing

class CVRP:
    def __init__(self, numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix):
        self.numberOfCustomers = numberOfCustomers
        self.numberOfVehicles  = numberOfVehicles
        self.capacityOfVehicle = capacityOfVehicle
        self.demandOfCustomers = demandOfCustomers
        self.costMatrix        = costMatrix
        self.initializeLP()
    
    def initializeLP(self):
        self.cvrpLP = pulp.LpProblem("CVRP", pulp.LpMinimize)
        objective = None
        x,y = [], []
        constraint1 = None
        
        # objective function and variables
        for i in range(len(costMatrix)): #adding depot
            xTemp1 = []
            for j in range(len(costMatrix)):
#                 if(i != j):
                xTemp2 = pulp.LpVariable('x('+str(i)+','+str(j)+')', cat='Binary')
                xTemp1.append(xTemp2)
#                 if(i != j):
                objective += xTemp2 * costMatrix[i][j]
#                 else:
#                     xTemp1.append(None)
            x.append(xTemp1) 
        self.cvrpLP += objective
        
        for i in range(len(costMatrix)):
            y.append(pulp.LpVariable('y'+str(i), lowBound=0, cat='Continuous'))
        
        
        # constraints
        # ensure that all customers are visited exactly once
        for i in range(1, len(costMatrix) - 1): #adding depot
            constraint1 = None
            for j in range(1, len(costMatrix)):
                if(i != j):
                    constraint1 += x[i][j]
            self.cvrpLP += constraint1 == 1
        
        # limits the maximum number of routes to the number of vehicles
        constraint2 = None
        for j in range(1, len(costMatrix) - 1): #not include depot
            constraint2 += x[0][j]
        self.cvrpLP += constraint2 <= self.numberOfVehicles
        
        # ensure together that the vehicle capacity is not exceeded
        for i in range(len(costMatrix)):
            constarint3a, constarint3b  = None, None
#             constarint3a = self.demandOfCustomers[i] <= y[i] 
            constarint3b = y[i] <= self.capacityOfVehicle
#             self.cvrpLP += constarint3a
            self.cvrpLP += constarint3b
        
        # ensure together that the vehicle capacity is not exceeded
        for i in range(len(costMatrix)): #adding depot
            for j in range(len(costMatrix)):
                constraint4 = None
#                 if(i != j):
                constraint4 += y[j] >= y[i] + self.demandOfCustomers[j]*x[i][j] - self.capacityOfVehicle*(1-x[i][j])
                self.cvrpLP += constraint4
        
        #guarantee the correct flow of vehicles through the arcs, by stating that if a vehicle arrives to a node
        #then it must depart from this node
        for h in range(1, len(costMatrix) - 1):
            constraint5a, constraint5b = None, None
            for i in range(0, len(costMatrix) - 1):
                if(i != h):
                    constraint5a += x[i][h]
            for j in range(1, len(costMatrix)):
                if(j != h):
                    constraint5b += x[h][j]
            self.cvrpLP += constraint5a - constraint5b == 0
        
        print(self.cvrpLP)
        
    def solve(self):
        status = self.cvrpLP.solve()
        print(pulp.LpStatus[self.cvrpLP.status])
    
    def getResult(self):
        print("Objective value: ", pulp.value(self.cvrpLP.objective))
        for v in self.cvrpLP.variables():
            print(v.name, " = ", v.varValue)
        
        


In [383]:
lp = CVRP(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix)

CVRP:
MINIMIZE
999999.0*x(0,0) + 404.0*x(0,1) + 307.0*x(0,2) + 621.0*x(0,3) + 118.0*x(0,4) + 338.0*x(0,5) + 404.0*x(1,0) + 999999.0*x(1,1) + 330.0*x(1,2) + 924.0*x(1,3) + 523.0*x(1,4) + 733.0*x(1,5) + 342.0*x(1,6) + 307.0*x(2,0) + 330.0*x(2,1) + 999999.0*x(2,2) + 613.0*x(2,3) + 396.0*x(2,4) + 524.0*x(2,5) + 263.0*x(2,6) + 621.0*x(3,0) + 924.0*x(3,1) + 613.0*x(3,2) + 999999.0*x(3,3) + 566.0*x(3,4) + 391.0*x(3,5) + 651.0*x(3,6) + 118.0*x(4,0) + 523.0*x(4,1) + 396.0*x(4,2) + 566.0*x(4,3) + 999999.0*x(4,4) + 234.0*x(4,5) + 182.0*x(4,6) + 338.0*x(5,0) + 733.0*x(5,1) + 524.0*x(5,2) + 391.0*x(5,3) + 234.0*x(5,4) + 999999.0*x(5,5) + 395.0*x(5,6) + 342.0*x(6,1) + 263.0*x(6,2) + 651.0*x(6,3) + 182.0*x(6,4) + 395.0*x(6,5) + 999999.0*x(6,6) + 0.0
SUBJECT TO
_C1: x(1,2) + x(1,3) + x(1,4) + x(1,5) + x(1,6) = 1

_C2: x(2,1) + x(2,3) + x(2,4) + x(2,5) + x(2,6) = 1

_C3: x(3,1) + x(3,2) + x(3,4) + x(3,5) + x(3,6) = 1

_C4: x(4,1) + x(4,2) + x(4,3) + x(4,5) + x(4,6) = 1

_C5: x(5,1) + x(5,2) + x(5,3) + 

In [384]:
lp.solve()
lp.getResult()

Optimal
Objective value:  2903.0
x(0,0)  =  0.0
x(0,1)  =  1.0
x(0,2)  =  0.0
x(0,3)  =  0.0
x(0,4)  =  1.0
x(0,5)  =  1.0
x(0,6)  =  0.0
x(1,0)  =  0.0
x(1,1)  =  0.0
x(1,2)  =  0.0
x(1,3)  =  0.0
x(1,4)  =  0.0
x(1,5)  =  0.0
x(1,6)  =  1.0
x(2,0)  =  0.0
x(2,1)  =  0.0
x(2,2)  =  0.0
x(2,3)  =  0.0
x(2,4)  =  0.0
x(2,5)  =  0.0
x(2,6)  =  1.0
x(3,0)  =  0.0
x(3,1)  =  0.0
x(3,2)  =  0.0
x(3,3)  =  0.0
x(3,4)  =  0.0
x(3,5)  =  0.0
x(3,6)  =  1.0
x(4,0)  =  0.0
x(4,1)  =  0.0
x(4,2)  =  1.0
x(4,3)  =  0.0
x(4,4)  =  0.0
x(4,5)  =  0.0
x(4,6)  =  0.0
x(5,0)  =  0.0
x(5,1)  =  0.0
x(5,2)  =  0.0
x(5,3)  =  1.0
x(5,4)  =  0.0
x(5,5)  =  0.0
x(5,6)  =  0.0
x(6,0)  =  0.0
x(6,1)  =  0.0
x(6,2)  =  0.0
x(6,3)  =  0.0
x(6,4)  =  0.0
x(6,5)  =  0.0
x(6,6)  =  0.0
y0  =  0.0
y1  =  6.0
y2  =  6.0
y3  =  6.0
y4  =  1.0
y5  =  5.0
y6  =  6.0


#### TESTING

In [387]:
numberOfCustomers = 31
capacityOfVehicle = 100
numberOfVehicles = 5
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {0:0,1:19,2:21,3:6,4:19,5:7,6:12,7:16,8:6,9:16,10:8,11:14,12:21,13:16,14:3,15:22,16:18,17:19,18:1,19:24,20:8,21:12,22:4,23:8,24:24,25:24,26:2,27:20,28:15,29:2,30:14,31:9}
# demandOfCustomers[0] = 0
demandOfCustomers[numberOfCustomers+1] = 0

In [388]:
xCoordinates = [82,96,50,49,13,29,58,84,14,2,3,5,98,84,61,1,88,91,19,93,50,98,5,42,61,9,80,57,23,20,85,98,82]
yCoordinates = [76,44,5,8,7,89,30,39,24,39,82,10,52,25,59,65,51,2,32,3,93,14,42,9,62,97,55,69,15,70,60,5,76]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 999999

In [391]:
len(costMatrix[0])

33

In [392]:
lp = CVRP(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix)

CVRP:
MINIMIZE
999999.0*x(0,0) + 34.0*x(0,1) + 79.0*x(0,10) + 101.0*x(0,11) + 28.0*x(0,12) + 51.0*x(0,13) + 27.0*x(0,14) + 81.0*x(0,15) + 25.0*x(0,16) + 74.0*x(0,17) + 76.0*x(0,18) + 73.0*x(0,19) + 77.0*x(0,2) + 36.0*x(0,20) + 64.0*x(0,21) + 84.0*x(0,22) + 78.0*x(0,23) + 25.0*x(0,24) + 75.0*x(0,25) + 21.0*x(0,26) + 25.0*x(0,27) + 84.0*x(0,28) + 62.0*x(0,29) + 75.0*x(0,3) + 16.0*x(0,30) + 72.0*x(0,31) + 97.0*x(0,4) + 54.0*x(0,5) + 51.0*x(0,6) + 37.0*x(0,7) + 85.0*x(0,8) + 88.0*x(0,9) + 34.0*x(1,0) + 999999.0*x(1,1) + 100.0*x(1,10) + 97.0*x(1,11) + 8.0*x(1,12) + 22.0*x(1,13) + 38.0*x(1,14) + 97.0*x(1,15) + 10.0*x(1,16) + 42.0*x(1,17) + 77.0*x(1,18) + 41.0*x(1,19) + 60.0*x(1,2) + 67.0*x(1,20) + 30.0*x(1,21) + 91.0*x(1,22) + 64.0*x(1,23) + 39.0*x(1,24) + 101.0*x(1,25) + 19.0*x(1,26) + 46.0*x(1,27) + 78.0*x(1,28) + 80.0*x(1,29) + 59.0*x(1,3) + 19.0*x(1,30) + 39.0*x(1,31) + 34.0*x(1,32) + 90.0*x(1,4) + 80.0*x(1,5) + 40.0*x(1,6) + 13.0*x(1,7) + 84.0*x(1,8) + 94.0*x(1,9) + 79.0*x(10,0) + 100.0

In [393]:
lp.solve()

KeyboardInterrupt: 

In [ ]:
lp.getResult()

### TESTING 2

In [10]:
numberOfCustomers = 15
capacityOfVehicle = 35
numberOfVehicles = 8
C = [i for i in range(1, numberOfCustomers+1)] #set of customers
V = [0] + C + [numberOfCustomers+1] #depot + customer nodes
demandOfCustomers = {0:0,1:19,2:30,3:16,4:23,5:11,6:31,7:15,8:28,9:8,10:8,11:7,12:14,13:6,14:19,15:11}
# demandOfCustomers[0] = 0
demandOfCustomers[numberOfCustomers+1] = 0

In [11]:
xCoordinates = [30,37,49,52,31,52,42,52,57,62,42,27,43,58,58,37,30]
yCoordinates = [40,52,49,64,62,33,41,41,58,42,57,68,67,48,27,69,40]

# Cost matrix
costMatrix = np.ndarray(shape=(len(V),len(V)))
for i in range(len(V)):
    for j in range(len(V)):
        if(i == 0 and j == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(j == 0 and i == len(V)-1):
            costMatrix[i][j] = 0
            continue
        
        if(i!=j):
            costMatrix[i][j] = int(distance.euclidean([xCoordinates[i],yCoordinates[i]], [xCoordinates[j],yCoordinates[j]]))
        else:
            costMatrix[i][j] = 999999

In [12]:
lp = CVRP(numberOfCustomers, numberOfVehicles, capacityOfVehicle, demandOfCustomers, costMatrix)

CVRP:
MINIMIZE
999999.0*x(0,0) + 13.0*x(0,1) + 20.0*x(0,10) + 28.0*x(0,11) + 29.0*x(0,12) + 29.0*x(0,13) + 30.0*x(0,14) + 29.0*x(0,15) + 21.0*x(0,2) + 32.0*x(0,3) + 22.0*x(0,4) + 23.0*x(0,5) + 12.0*x(0,6) + 22.0*x(0,7) + 32.0*x(0,8) + 32.0*x(0,9) + 13.0*x(1,0) + 999999.0*x(1,1) + 7.0*x(1,10) + 18.0*x(1,11) + 16.0*x(1,12) + 21.0*x(1,13) + 32.0*x(1,14) + 17.0*x(1,15) + 13.0*x(1,16) + 12.0*x(1,2) + 19.0*x(1,3) + 11.0*x(1,4) + 24.0*x(1,5) + 12.0*x(1,6) + 18.0*x(1,7) + 20.0*x(1,8) + 26.0*x(1,9) + 20.0*x(10,0) + 7.0*x(10,1) + 999999.0*x(10,10) + 18.0*x(10,11) + 10.0*x(10,12) + 18.0*x(10,13) + 34.0*x(10,14) + 13.0*x(10,15) + 20.0*x(10,16) + 10.0*x(10,2) + 12.0*x(10,3) + 12.0*x(10,4) + 26.0*x(10,5) + 16.0*x(10,6) + 18.0*x(10,7) + 15.0*x(10,8) + 25.0*x(10,9) + 28.0*x(11,0) + 18.0*x(11,1) + 18.0*x(11,10) + 999999.0*x(11,11) + 16.0*x(11,12) + 36.0*x(11,13) + 51.0*x(11,14) + 10.0*x(11,15) + 28.0*x(11,16) + 29.0*x(11,2) + 25.0*x(11,3) + 7.0*x(11,4) + 43.0*x(11,5) + 30.0*x(11,6) + 36.0*x(11,7) + 31.

In [13]:
lp.solve()

Optimal


In [14]:
lp.getResult()

Objective value:  444.0
x(0,0)  =  0.0
x(0,1)  =  1.0
x(0,10)  =  1.0
x(0,11)  =  1.0
x(0,12)  =  0.0
x(0,13)  =  1.0
x(0,14)  =  1.0
x(0,15)  =  0.0
x(0,16)  =  0.0
x(0,2)  =  1.0
x(0,3)  =  1.0
x(0,4)  =  0.0
x(0,5)  =  0.0
x(0,6)  =  1.0
x(0,7)  =  0.0
x(0,8)  =  0.0
x(0,9)  =  0.0
x(1,0)  =  0.0
x(1,1)  =  0.0
x(1,10)  =  0.0
x(1,11)  =  0.0
x(1,12)  =  0.0
x(1,13)  =  0.0
x(1,14)  =  0.0
x(1,15)  =  0.0
x(1,16)  =  1.0
x(1,2)  =  0.0
x(1,3)  =  0.0
x(1,4)  =  0.0
x(1,5)  =  0.0
x(1,6)  =  0.0
x(1,7)  =  0.0
x(1,8)  =  0.0
x(1,9)  =  0.0
x(10,0)  =  0.0
x(10,1)  =  0.0
x(10,10)  =  0.0
x(10,11)  =  0.0
x(10,12)  =  1.0
x(10,13)  =  0.0
x(10,14)  =  0.0
x(10,15)  =  0.0
x(10,16)  =  0.0
x(10,2)  =  0.0
x(10,3)  =  0.0
x(10,4)  =  0.0
x(10,5)  =  0.0
x(10,6)  =  0.0
x(10,7)  =  0.0
x(10,8)  =  0.0
x(10,9)  =  0.0
x(11,0)  =  0.0
x(11,1)  =  0.0
x(11,10)  =  0.0
x(11,11)  =  0.0
x(11,12)  =  0.0
x(11,13)  =  0.0
x(11,14)  =  0.0
x(11,15)  =  0.0
x(11,16)  =  0.0
x(11,2)  =  0.0
x(11,3